### DS Machine Learning - Amir Chebli - GLSI3 - G1

# Import des bibliothèques nécessaires

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score, precision_score, f1_score, confusion_matrix, classification_report

# Identification et éxplication du problème d'apprentissage

D'après les informations disponibles, il semble que le Dataset DS1 soit lié à des informations sur les utilisateurs, notamment le sexe, l'âge, le salaire estimé et une variable de résultat binaire "Purchased", qui indique si un utilisateur a effectué un achat ou non. L'objectif du problème d'apprentissage est de prédire si un utilisateur va effectuer un achat sur la base des caractéristiques données.

Il s'agit d'un problème de classification binaire. La tâche consiste à classer chaque instance (utilisateur) dans l'une des deux classes suivantes : "Purchased" (classe positive) ou "Not Purchased" (classe négative). Les caractéristiques (Gender, Age, Estimated Salary) sont utilisées pour faire cette prédiction.

Les algorithmes d'apprentissage automatique les plus courants pour les problèmes de classification binaire comprennent la régression logistique, la forêt aléatoire ainsi que les modèles de boosting de gradient (par exemple XGBoost). 

# Partie I : Prétraitement des données

### Chargement des données depuis le fichier CSV

In [2]:
file_path = "DS1-L.csv"
DS1_data = pd.read_csv(file_path)

### Exploratin des données

In [3]:
DS1_data.head() # Afficher les premières lignes du DataFrame

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [4]:
DS1_data.info() # Informations sur les colonnes et les types de données

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   User ID          400 non-null    int64 
 1   Gender           400 non-null    object
 2   Age              400 non-null    int64 
 3   EstimatedSalary  400 non-null    int64 
 4   Purchased        400 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 15.8+ KB


In [5]:
# Suppression des colonnes inutiles
columns_to_drop = ['User ID']
DS1_data = DS1_data.drop(columns=columns_to_drop, axis=1)

In [6]:
DS1_data.head() # Afficher les premières lignes du DataFrame après suppression

,Gender,Age,EstimatedSalary,Purchased
0,Male,19,19000,0
1,Male,35,20000,0
2,Female,26,43000,0
3,Female,27,57000,0
4,Male,19,76000,0


In [7]:
# Rendre les données plus unifromes (encodage)
label_encoder = LabelEncoder() # Encodage de la colonne 'Sex'
DS1_data['Gender'] = label_encoder.fit_transform(DS1_data['Gender'])
DS1_data.head()  # Afficher les premières lignes du DataFrame après l'encodage

,Gender,Age,EstimatedSalary,Purchased
0,1,19,19000,0
1,1,35,20000,0
2,0,26,43000,0
3,0,27,57000,0
4,1,19,76000,0


In [8]:
# Affichage des décomptes de valeurs uniques
for column in DS1_data.columns:
    print(f"Nombre de valeurs uniques pour la colonne {column}: {DS1_data[column].nunique()}")

Nombre de valeurs uniques pour la colonne Gender: 2
Nombre de valeurs uniques pour la colonne Age: 43
Nombre de valeurs uniques pour la colonne EstimatedSalary: 117
Nombre de valeurs uniques pour la colonne Purchased: 2


### Traitement des données manquantes

In [9]:
# Afficher la somme des valeurs manquantes avant le traitement
DS1_data.isnull().sum()

Gender             0
Age                0
EstimatedSalary    0
Purchased          0
dtype: int64

Puisqu'il n'y a pas de données manquantes, aucun traitement ne sera nécessaire dans cette partie.

### Feature Scaling

In [10]:
# Colonnes pour le Feature Scaling
colonnes_numeriques = ['Age', 'EstimatedSalary'] # Ce sont les colonnes ayant des données qui ont des échelles différentes
# Appliquer le Feature Scaling
mise_a_lechelle_numerique = StandardScaler() 
DS1_data[colonnes_numeriques] = mise_a_lechelle_numerique.fit_transform(DS1_data[colonnes_numeriques])

In [11]:
DS1_data.head()  # Afficher les premières lignes du DataFrame après le Feature Scaling

,Gender,Age,EstimatedSalary,Purchased
0,1,-1.781797,-1.490046,0
1,1,-0.253587,-1.460681,0
2,0,-1.113206,-0.785290,0
3,0,-1.017692,-0.374182,0
4,1,-1.781797,0.183751,0


### Séparation des données

In [12]:
# Séparation des données en features (X) et target variable (y)
X = DS1_data.drop(['Purchased'], axis=1)
y = DS1_data['Purchased']

In [13]:
# Split des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Partie II : Implémentation des modèles d'apprentissage

### Régression logistique

In [14]:
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)
logistic_predictions = logistic_model.predict(X_test)

### Forêt aléatoire

In [15]:
random_forest_model = RandomForestClassifier()
random_forest_model.fit(X_train, y_train)
random_forest_predictions = random_forest_model.predict(X_test)

### XGBoost

In [16]:
xgboost_model = xgb.XGBClassifier()
xgboost_model.fit(X_train, y_train)
xgboost_predictions = xgboost_model.predict(X_test)

# Partie III : Analyse des résultats

### Création d'une procédure pour l'évaluation des modéles

In [17]:
def evaluate_model(predictions, model_name):
    print(f"--------- {model_name} ---------")
    print(f"Accuracy: {accuracy_score(y_test, predictions)}")
    print(f"Precision: {precision_score(y_test, predictions)}")
    print(f"F-Measure: {f1_score(y_test, predictions)}")
    print("Matrice de confusion:\n", confusion_matrix(y_test, predictions))
    print("Rapport de classification:\n", classification_report(y_test, predictions))
    print("\n")

### Comparaison des modèles

In [18]:
evaluate_model(logistic_predictions, "Régression Logistique")
evaluate_model(random_forest_predictions, "Forêt Aléatoire")
evaluate_model(xgboost_predictions, "XGBoost")

--------- Régression Logistique ---------
Accuracy: 0.8875
Precision: 0.9130434782608695
F-Measure: 0.8235294117647057
Matrice de confusion:
 [[50  2]
 [ 7 21]]
Rapport de classification:
               precision    recall  f1-score   support

           0       0.88      0.96      0.92        52
           1       0.91      0.75      0.82        28

    accuracy                           0.89        80
   macro avg       0.90      0.86      0.87        80
weighted avg       0.89      0.89      0.88        80



--------- Forêt Aléatoire ---------
Accuracy: 0.925
Precision: 0.8928571428571429
F-Measure: 0.8928571428571429
Matrice de confusion:
 [[49  3]
 [ 3 25]]
Rapport de classification:
               precision    recall  f1-score   support

           0       0.94      0.94      0.94        52
           1       0.89      0.89      0.89        28

    accuracy                           0.93        80
   macro avg       0.92      0.92      0.92        80
weighted avg       0.93     

### Conclusion

Compte tenu de ces résultats, le modèle Random Forest semble être le meilleur choix pour cet ensemble de données et ce problème spécifique. Il présente la plus grande accuracy et un bon équilibre entre la précision et le rappel. Le modèle Random Forest est une méthode d'ensemble qui combine les prédictions de plusieurs arbres de décision, ce qui permet souvent d'obtenir des modèles robustes et précis.
